In [1]:
import numpy as np
import re
import itertools
from sklearn import svm

In [2]:
def pre_processing(file):

    with open(file) as f:
        lines = f.readlines()
    f.close()
    
    lines = [x.strip() for x in lines] 
    N = len(lines)

    X = np.zeros((N, 46), dtype=float)
    Y = np.zeros((N, 2), dtype=float)

    for i in range(len(lines)):

        for m in re.finditer('\d*\d[:][+-]?([0-9]*[.])?[0-9]+', lines[i]):
            sign = lines[i][m.start():m.end()].find(':')
            X[i][np.int(lines[i][m.start():m.end()][:sign])-1] = np.float(lines[i][m.start():m.end()][sign+1:]) 
        
        Y[i][0] = np.int(lines[i][0])
        Y[i][1] = np.int(re.search(":\d*", lines[i]).group(0)[1:])
    
    return X, Y

In [3]:
def transform_pairwise(X, Y):
    
    X_tp, y_tp = list(), list()
    row = 0
    
    while row < X.shape[0]:

        start, end = row, row
        while Y[start][1] == Y[end][1]:
            end += 1
            row += 1
            if end == Y.shape[0]:
                break

        if end - start > 1:
            comb = itertools.combinations(range(start, end), 2)
            for _, (i, j) in enumerate(comb):
                if Y[i][0] != Y[j][0]:
                    X_tp.append(X[i] - X[j])
                    y_tp.append(np.sign(Y[i][0] - Y[j][0]))
                    
    for i in range(len(y_tp)):
        if y_tp[i] != (-1) ** i:
            X_tp[i] = -X_tp[i]
            y_tp[i] = -y_tp[i]
            
    return np.asarray(X_tp), np.asarray(y_tp)

In [4]:
class Graph: 
    def __init__(self,num): 
        self.graph = {i:[] for i in range(num)}
  
    def add_edge(self,u,v): 
        self.graph[u].append(v) 
  
    def topological_util(self, v, visited, stack): 
        
        visited[v] = True
        for i in self.graph[v]: 
            if visited[i] == False: 
                self.topological_util(i,visited,stack) 
  
        stack.insert(0,v) 
  

    def topological_sort(self): 
        
        visited = [False]*len(list(self.graph))
        stack =[] 

        for i in range(len(list(self.graph))): 
            if visited[i] == False: 
                self.topological_util(i,visited,stack) 
  
        return stack

In [5]:
def NDCG5(model, X, Y):
    
    row = 0
    total_nDCG = 0
    count = 0
    
    while row < X.shape[0]:

        start, end = row, row
        while Y[start][1] == Y[end][1]:
            end += 1
            row += 1
            if end == Y.shape[0]:
                break
        
        if end - start > 1:
            doc_q, r = X[start:end], Y[start:end][:,0]
            comb = itertools.combinations(range(len(doc_q)), 2)
            graph = Graph(len(doc_q))
            
            for _, (i, j) in enumerate(comb):
                sign = model.predict((doc_q[i] - doc_q[j]).reshape(1, -1))
                if sign[0] == 1:
                    graph.add_edge(i, j)
                else:
                    graph.add_edge(j, i)
            
            ans_q = graph.topological_sort()[:5]    
            DCG = r[ans_q[0]]
            
            for k in range(1, len(ans_q)):
                DCG += r[ans_q[k]] / np.log2(1+k)
            
            ideal_order = sorted(r, reverse=True)[:5]
            IDCG = ideal_order[0]
            
            for i in range(1, len(ideal_order)):
                IDCG += ideal_order[i]/np.log2(1+i)
            
            if IDCG != 0:
                count += 1
                total_nDCG += (DCG / IDCG)
    
    if count > 0:
        total_nDCG /= count
    
    return total_nDCG

# Training & Validation

In [6]:
X, Y = pre_processing("data/train.txt")
X_train, y_train = transform_pairwise(X, Y)
X_val, Y_val = pre_processing("data/vali.txt")

In [20]:
for k in range(-10, 10):
    reg = 2 ** k
    linear_svm = svm.LinearSVC(C=reg, max_iter=4000)
    linear_svm.fit(X_train, y_train)
    print("NDCG@5=", NDCG5(linear_svm, X_val, Y_val), ", reg param=", reg)

NDCG@5= 0.6812444465646103 , reg param= 0.0009765625
NDCG@5= 0.682034653799786 , reg param= 0.001953125
NDCG@5= 0.6836987341763024 , reg param= 0.00390625
NDCG@5= 0.6851905151163139 , reg param= 0.0078125
NDCG@5= 0.6880043093377225 , reg param= 0.015625
NDCG@5= 0.688713987016742 , reg param= 0.03125
NDCG@5= 0.6901058807325507 , reg param= 0.0625
NDCG@5= 0.6902232401656424 , reg param= 0.125
NDCG@5= 0.6905462531674014 , reg param= 0.25
NDCG@5= 0.690586576784637 , reg param= 0.5
NDCG@5= 0.6891097764466372 , reg param= 1
NDCG@5= 0.6859355183227771 , reg param= 2
NDCG@5= 0.6827332899706711 , reg param= 4


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NDCG@5= 0.6819465120086807 , reg param= 8


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NDCG@5= 0.6837409976673199 , reg param= 16


...

NDCG@5= 0.6842526402993285 , reg param= 128


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NDCG@5= 0.6651102482719813 , reg param= 256


//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NDCG@5= 0.6033815899548614 , reg param= 512


In [24]:
for _ in range(10):
    parm = np.random.uniform(0.25, 1)
    linear_svm = svm.LinearSVC(C=parm)
    linear_svm.fit(X_train, y_train)
    print("NDCG@5=", NDCG5(linear_svm, X_val, Y_val), ", reg param=", parm)

NDCG@5= 0.6892224311580171 , reg param= 0.850781952259728
NDCG@5= 0.6892224311580171 , reg param= 0.6889684439796843
NDCG@5= 0.690586576784637 , reg param= 0.4561258313781682
NDCG@5= 0.6892224311580171 , reg param= 0.8199644648650208
NDCG@5= 0.6892224311580171 , reg param= 0.854878680701946
NDCG@5= 0.6892224311580171 , reg param= 0.7883337086444778
NDCG@5= 0.6892224311580171 , reg param= 0.6898502408626326
NDCG@5= 0.690586576784637 , reg param= 0.35109277470240063
NDCG@5= 0.690586576784637 , reg param= 0.42082429817673594
NDCG@5= 0.6892224311580171 , reg param= 0.8764932553944303


In [22]:
best_reg = 0.456
best_model = svm.LinearSVC(C=best_reg, max_iter=4000)
best_model.fit(X_train, y_train)

LinearSVC(C=0.449, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=4000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [23]:
NDCG5(best_model, X_val, Y_val)

0.690586576784637

# Test

In [25]:
X_test, Y_test = pre_processing("data/test.txt")
NDCG5(best_model, X_test, Y_test)

0.6770960130026611